# 经典策略模式

In [4]:

"""
假如一个网店制定了下述折扣规则。
有 1000 或以上积分的顾客， 每个订单享 5% 折扣。
同一订单中， 单个商品的数量达到 20 个或以上， 享 10% 折扣。
订单中的不同商品达到 10 个或以上， 享 7% 折扣。
简单起见， 我们假定一个订单一次只能享用一个折扣
"""


from abc import ABC, abstractmethod
from collections import namedtuple

# 顾客名称和积分
Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """xxx"""


class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0



joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]


print("开始使用不同打折策略：")
print("joe","--FidelityPromo--",Order(joe, cart, FidelityPromo()))
print("ann","--FidelityPromo--",Order(ann, cart, FidelityPromo()))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print("banana_cart","--BulkItemPromo--",Order(joe, banana_cart, BulkItemPromo()));


long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print("long_order","--LargeOrderPromo--",Order(joe, long_order, LargeOrderPromo()))

开始使用不同打折策略：
joe --FidelityPromo-- <Order total: 42.00 due: 42.00>
ann --FidelityPromo-- <Order total: 42.00 due: 21.00>
banana_cart --BulkItemPromo-- <Order total: 30.00 due: 28.50>
long_order --LargeOrderPromo-- <Order total: 10.00 due: 9.30>


# 使用函数实现“策略”模式

In [7]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0


def xxxxxxx_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 10
    return 0

# 全部策略列表
# 手动添加新的定义策略函数，然后手动维护到promos列表，麻烦
# promos = [fidelity_promo, bulk_item_promo, large_order_promo]

# 利用globals()返回字典中的各个名字，只选择以_promo结尾的名称，并过滤掉best_promo
# 省掉了手动维护promos列表的工作，但是需要在策略函数定义命名上进行约束
promos = [
    globals()[name] for name in globals()
    if name.endswith("_promo") and name != "best_promo"
]

def best_promo(order):
    """
    选择最佳折扣
    """
    return max(promo(order) for promo in promos)


joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]

print("开始使用不同打折策略：")
print("joe", "--FidelityPromo--", Order(joe, cart, fidelity_promo))
print("ann", "--FidelityPromo--", Order(ann, cart, fidelity_promo))

banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print("banana_cart", "--BulkItemPromo--", Order(joe, banana_cart, bulk_item_promo))
#
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print("long_order", "--LargeOrderPromo--", Order(joe, long_order, large_order_promo))


print("开始使用best打折策略：")
print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

开始使用不同打折策略：
joe --FidelityPromo-- <Order total: 42.00 due: 42.00
ann --FidelityPromo-- <Order total: 42.00 due: 39.90
banana_cart --BulkItemPromo-- <Order total: 30.00 due: 28.50
long_order --LargeOrderPromo-- <Order total: 10.00 due: 9.30
开始使用best打折策略：
<Order total: 10.00 due: 9.30
<Order total: 30.00 due: 28.50
<Order total: 42.00 due: 39.90


> promos = [func for name, func in
inspect.getmembers(promotions, inspect.isfunction)]  promos列表新的维护办法：在一个单独的模块(promotions)中保存所有策略,然后倒入promotions模块，利用内省函数的inspect模块找到promotions模块内部全部
函数的名称和函数对象

# 命令模式